## Telegram Scrape Test


In [13]:
import sys
import asyncio
from pathlib import Path

proj = Path().resolve().parent
if str(proj) not in sys.path:
    sys.path.append(str(proj))

from src.collector import collect_From_channes

channels = [
    "https://t.me/lobelia4cosmetics",
    "https://t.me/tikvahpharma",
]

async def collect_channels(channel_list):
    for channel in channel_list:
        username = channel.split("/")[-1]
        await collect_From_channes(username)

await collect_channels(channels)
print("Done scraping and saving!")

Scraping messages from lobelia4cosmetics
1. SARDINES **
Price 900 birr 
Telegram :-****@Lobeliacosmetics****
Msg👉 Lobelia pharmacy and cosmetics 
☎️ call ****0911562031****/0911587703
Adress:- Infront of Bole Medhanialem high school 
ALEMNESH PLAZA GROUND FLOOR 
Open Monday - Monday from 8am until midnight ከሰኞ - እስከ ሰኞ  ከጧቱ 2:00 ስዓት - እስከ ምሽቱ 6:00 ሰዓት
🏍🏍🏍 የትራንስፖርት አማራጭ ከ70 - 500 ብር
For delivery option fees are from 70 birr -500 birr**

2. IRON KIDS OMEGA3
Price 4500 birr 
Telegram https://t.me/lobelia4cosmetics
Msg👉 Lobelia pharmacy and cosmetics 
☎️ call 0911562031/0911587703
Adress:- Infront of Bole Medhanialem high school 
Open Monday - Monday from 8am until midnight ከሰኞ - እስከ ሰኞ  ከጧቱ 2:00 ስዓት - እስከ ምሽቱ 6:00 ሰዓት
🏍🏍🏍 የትራንስፖርት አማራጭ ከ70 - 200 ብር
For delivery option fees are from 70 birr - 200 birr

4. DAY AND NIGHT QUIL 48 TABLETS 
Price 4500 birr 
Telegram https://t.me/lobelia4cosmetics
Msg👉 Lobelia pharmacy and cosmetics 
☎️ call 0911562031/0911587703
Adress:- Infront of Bole Medhani

In [11]:
print(f"Scraped {len(all_data)} messages from {channel_username}")


NameError: name 'all_data' is not defined

In [3]:
from telethon.sync import TelegramClient
from pathlib import Path
from datetime import datetime
import json

# Replace these with your actual credentials
api_id = 23969142
api_hash = "63cfef12e419c6f4a413724fccd8dda9"


client = TelegramClient('session', api_id, api_hash)

async def collect_from_channel(channel_username):
    await client.start()
    print(f"📥 Scraping messages from {channel_username}...")

    all_data = []

    try:
        async for msg in client.iter_messages(channel_username, limit=100):
            if msg.text:
                all_data.append({
                    "id": msg.id,
                    "date": msg.date.strftime("%Y-%m-%d %H:%M:%S"),
                    "text": msg.text
                })

        print(f"✅ Scraped {len(all_data)} messages.")

        if not all_data:
            print("⚠️ No messages found — nothing to save.")
            return

        folder = Path("data/raw") / datetime.now().strftime("%Y-%m-%d")
        folder.mkdir(parents=True, exist_ok=True)
        out_file = folder / f"{channel_username}.json"

        with open(out_file, "w", encoding="utf-8") as f:
            json.dump(all_data, f, indent=2, ensure_ascii=False)
        print(f"✅ Saved to {out_file}")

    except Exception as e:
        print(f"❌ Error while scraping or saving: {e}")

    finally:
        await client.disconnect()

async def main():
    await collect_from_channel("lobelia4cosmetics")
    await collect_from_channel("tikvahpharma")

await main()
print("Done scraping and saving!")

📥 Scraping messages from lobelia4cosmetics...
✅ Scraped 98 messages.
✅ Saved to data/raw/2025-07-15/lobelia4cosmetics.json
📥 Scraping messages from tikvahpharma...
✅ Scraped 92 messages.
✅ Saved to data/raw/2025-07-15/tikvahpharma.json
Done scraping and saving!
